<a href="https://colab.research.google.com/github/MasterDiaz410/ProyectoSL1/blob/master/Proyecto1_Parte_2_SL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 1 parte 2

In [1]:

import numpy as np
import pandas as pd
from joblib import load
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from scipy.stats import norm
import math


In [2]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

In [3]:
#montamos el contenido del drive
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Cargando la informacion desde el drive**

In [4]:
# informacion generada por la parte 1 del proyecto

modelTree = load('gdrive/My Drive/Colab Notebooks/proyecto1_modelos/_20200630_050442_log_decision_tree.joblib') 

media = pd.read_csv('gdrive/My Drive/Colab Notebooks/proyecto1_modelos/_20200630_050442_log_bayes/media.csv')
desv = pd.read_csv('gdrive/My Drive/Colab Notebooks/proyecto1_modelos/_20200630_050442_log_bayes/desviacion.csv')
prob = pd.read_csv('gdrive/My Drive/Colab Notebooks/proyecto1_modelos/_20200630_050442_log_bayes/probabilidades.csv')
y_class = pd.read_csv('gdrive/My Drive/Colab Notebooks/proyecto1_modelos/_20200630_050442_log_bayes/class.csv')

modelNaivesBayes = {"mean":media, "stdev":desv, "probabilities":prob, "class":y_class}

xTestingData = pd.read_csv('gdrive/My Drive/Colab Notebooks/xTestingData.csv')
yTestingData = pd.read_csv('gdrive/My Drive/Colab Notebooks/yTestingData.csv')

# Funciones para realizar los calculos

In [5]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
def getArgMax(probability):
    max_value = 0
    argmax = -1
    for (key, value) in probability.items():
        if (key == 0):
            max_value = max(value)
            argmax = key
        else:
            tmp = max(value)
            if(max_value < tmp):
                max_value = tmp
                argmax = key
    return argmax

In [7]:
def predictBayes(model, xTestingData):
    y_predict = []
    for i in range(xTestingData.shape[0]):
        probability={}
        for y_class in model["class"].values:
            y_class = int(y_class)
            probability[y_class] = model["probabilities"].iloc[y_class][0]
            for index, _ in enumerate(xTestingData.iloc[i]):
                probability[y_class] *= norm.pdf(xTestingData.iloc[i], model["mean"].iloc[y_class, index], model["stdev"].iloc[y_class, index])
        y_predict.append(getArgMax(probability))
    return np.array(y_predict)

In [8]:
def getMetrics(y_true, y_predict):
    accuracy = accuracy_score(y_true, y_predict)
    error = mean_squared_error(y_true, y_predict)
    precision = precision_score(y_true, y_predict, average='weighted')
    recall = recall_score(y_true, y_predict, average='weighted')
    f1 = f1_score(y_true, y_predict, average="weighted")
    
    return np.array([accuracy, error, precision, recall, f1])

In [9]:
#segun definicion de wikipedia
def moda(data):
    repetitions = 0
    moda = -1
    for i in data:
        n = data.count(i)
        if n > repetitions:
            repetitions = n
    for i in data:
        n = data.count(i)
        if n == repetitions and moda == -1:
            moda = i     
    return moda

In [10]:
def get_final_prediction(prediction_joined):
    moda_prediction = []
    for predict in prediction_joined:
        moda_predict = moda(list(predict))
        moda_prediction.append(moda_predict)
    return np.array(moda_prediction)

In [11]:
x_test_ = xTestingData.drop(['Fare'], axis=1)

#Predicciones

In [12]:
y_predict_tree = modelTree.predict(xTestingData)
y_predict_bayes = predictBayes(modelNaivesBayes, xTestingData)
y_predict_joined = np.stack((y_predict_tree, y_predict_bayes), axis=-1)
y_hat_moda = get_final_prediction(y_predict_joined)

In [13]:
y_prediction_summary = np.stack((y_predict_tree, y_predict_bayes, y_hat_moda), axis=-1)
df_predictions = pd.DataFrame(y_prediction_summary, columns = ["Decision Tree","Bayes","Moda Predict"])
df_predictions.head(15)

,Decision Tree,Bayes,Moda Predict
0,0,1,0
1,1,1,1
2,1,0,1
3,0,0,0
4,1,0,1
5,0,0,0
6,0,0,0
7,0,1,0
8,1,0,1
9,1,0,1


# Métricas

In [14]:
metrics_tree = getMetrics(yTestingData, y_predict_tree)
metrics_bayes = getMetrics(yTestingData, y_predict_bayes)
metrics_moda_predicted = getMetrics(yTestingData, y_hat_moda)
metrics_joined = np.stack((metrics_tree, metrics_bayes, metrics_moda_predicted), axis=-1)

In [15]:
df_metrics = pd.DataFrame(metrics_joined, index = ["Accuracy","Error","Precision","Recall","F1"], 
                          columns = ["Decision Tree","Bayes","Moda Predict"])
df_metrics

,Decision Tree,Bayes,Moda Predict
Accuracy,0.873950,0.705882,0.873950
Error,0.126050,0.294118,0.126050
Precision,0.873040,0.695896,0.873040
Recall,0.873950,0.705882,0.873950
F1,0.872891,0.695160,0.872891


# Predicciones Probabilisticas de las proporciones para el modelo



**Naive Bayes**

In [16]:
modelNaivesBayes["probabilities"]

,probabilities
0,0.616466
1,0.383534


In [17]:
modelNaivesBayes["stdev"]

,Fare,passenger_sex_codes,passenger_class_codes
0,30.469067,0.356912,0.731240
1,58.172405,0.472018,0.858798


In [18]:
modelNaivesBayes["mean"]

,Fare,passenger_sex_codes,passenger_class_codes
0,22.065131,0.850163,0.456026
1,45.238896,0.335079,1.026178


# Grafo

In [19]:
# testee muchas formas pero no logre generar el grafo, al parecer es un bug con la version de python
# y colab que estoy usando 
# https://stackoverflow.com/questions/50103486/installation-graphviz-no-module-named-graphviz
def generateGraph(x,y):
    model = tree.DecisionTreeClassifier()
    model = model.fit(x, y)
    graphData = tree.export_graphviz(model, out_file=None,
                                      filled=True, rounded=True,
                                      class_names=["No vive","vive"],                                      
                                      special_characters=True)  
    graph = graphviz.Source(graphData)
    
    return graph 

x = xTestingData.head(25).values
y = y_hat_moda[0:25]

# tratamos de generar el grafo
generateGraph(x,y)   

NameError: ignored